In [963]:

import numpy as np
from sklearn.utils import resample

from dtree import *

class RandomForest621:
    def __init__(self, n_estimators=10, oob_score=False,min_samples_leaf=3,max_features=0.3):
        self.n_estimators = n_estimators
        self.oob_score = oob_score
        self.oob_score_ = np.nan
        self.min_samples_leaf=min_samples_leaf
        self.max_features=max_features
        self.trees=[]

    def compute_oob_score(self,X,y):

        oob_index_values={key:[] for key in range(len(X))}
        oob_index_preds={}
        
        if isinstance(self, RandomForestRegressor621):

            for tree in self.trees:
            
           

                oob_predictions_leaves=tree.predict(X[tree.oob_index])
                oob_predictions=[leaf.prediction for leaf in oob_predictions_leaves]

                for i in range(len(oob_predictions)):
                
                    index=tree.oob_index[i]

                    oob_index_values[index].append(oob_predictions[i])

        #loss = self.trees[0].loss
        #loss(y)
            for i in range(len(X)):
                oob_index_preds[i]=np.mean(oob_index_values[i])

            #Have to deal with missing values
            
            oob_predictions=list(oob_index_preds.values())
            oob_predictions=np.array(oob_predictions)
            oob_predictions=oob_predictions[np.where(~np.isnan(oob_predictions))[0]]
            y_filtered=y[np.where(~np.isnan(oob_predictions))[0]]
        #arr[np.where(~np.isnan(test_lis))[0]]


            return r2_score(y_filtered,oob_predictions)
        
        
        elif isinstance(self, RandomForestClassifier621):
            
                for tree in self.trees:
        
                    oob_predictions_leaves=tree.predict(X[tree.oob_index])
                    oob_predictions=[leaf.prediction[0] for leaf in oob_predictions_leaves]

                    for i in range(len(oob_predictions)):
                
                        index=tree.oob_index[i]

                        oob_index_values[index].append(oob_predictions[i])

        #loss = self.trees[0].loss
        #loss(y)
                for i in range(len(X)):
                    #stats.mode(rf.oob_score_[0])[0][0]
                    oob_index_preds[i]=stats.mode(oob_index_values[i])[0]

            #Have to deal with missing values
            
                oob_predictions=list(oob_index_preds.values())
                oob_predictions=np.concatenate(oob_predictions)
                #oob_predictions=[oob_predictions[k][0] for k in range(len(X_train))]
                oob_predictions=np.array(oob_predictions)
                oob_predictions=oob_predictions[np.where(~np.isnan(oob_predictions))[0]]
                y_filtered=y[np.where(~np.isnan(oob_predictions))[0]]
        #arr[np.where(~np.isnan(test_lis))[0]]
                #return oob_index_values
                #return oob_predictions
                return accuracy_score(y_filtered,oob_predictions)
    def fit(self, X, y):
        """
        Given an (X, y) training set, fit all n_estimators trees to different,
        bootstrapped versions of the training data.  Keep track of the indexes of
        the OOB records for each tree.  After fitting all of the trees in the forest,
        compute the OOB validation score estimate and store as self.oob_score_, to
        mimic sklearn.
        """

        oob_index_dict={}
        trees_dict={}

        for i in range(self.n_estimators):
            n = len(y)
            idx = np.random.randint(0, n, size = n)
            X_train = X[idx]
            y_train = y[idx]
            oob_index=np.array([i for i in range(len(X)) if i not in idx])
            #oob_index_dict[i]=oob_index
            if isinstance(self, RandomForestRegressor621):
                tree=RegressionTree621(self.min_samples_leaf, self.max_features, oob_index)
                tree.fit(X_train,y_train)
            #trees_dict[i]=tree
                self.trees.append(tree)
                if self.oob_score==True:
                    self.oob_score_=self.compute_oob_score(X,y)

            elif isinstance(self, RandomForestClassifier621):
                tree=ClassifierTree621(self.min_samples_leaf, self.max_features, oob_index)
                tree.fit(X_train,y_train)
                self.trees.append(tree)
                if self.oob_score==True:
                    self.oob_score_=self.compute_oob_score(X,y)
                #self.oob_score_=self.compute_oob_score(self,X,y)

class RandomForestRegressor621(RandomForest621):
    def __init__(self, n_estimators=10, min_samples_leaf=3, 
    max_features=0.3, oob_score=False):
        super().__init__(n_estimators, oob_score=oob_score, min_samples_leaf=min_samples_leaf, max_features=max_features)
        #self.trees = ...

    def predict(self, X_test) -> np.ndarray:
        """
        Given a 2D nxp array with one or more records, compute the weighted average
        prediction from all trees in this forest. Weight each trees prediction by
        the number of observations in the leaf making that prediction.  Return a 1D vector
        with the predictions for each input record of X_test.
        """
        
        prediction_leaves=[]
        weighted_predictions=[]
        count_leaves=[]

        for tree in self.trees:

            leaves=tree.predict(X_test)
            prediction_leaves.append(leaves)



        for i in range(self.n_estimators):

            weighted_predictions.append([leaf.prediction*leaf.n for leaf in prediction_leaves[i]])
            count_leaves.append([leaf.n for leaf in prediction_leaves[i]])

        weighted_arr=np.array(weighted_predictions)
        weighted_pred_sum=np.sum(weighted_arr,axis=0)
        count_leaves_arr=np.array(count_leaves)
        leaves_sum_count=np.sum(count_leaves_arr,axis=0)

        final_prediction_array=weighted_pred_sum/leaves_sum_count

        return final_prediction_array
    
      
    def score(self, X_test, y_test) -> float:
        """
        Given a 2D nxp X_test array and 1D nx1 y_test array with one or more records,
        collect the prediction for each record and then compute R^2 on that and y_test.
        """
        y_pred = self.predict(X_test)
        return r2_score(y_test, y_pred)
        
class RandomForestClassifier621(RandomForest621):
    def __init__(self, n_estimators=10, min_samples_leaf=3, 
    max_features=0.3, oob_score=False):
        super().__init__(n_estimators, oob_score=oob_score)
        n_estimators = n_estimators
        self.min_samples_leaf = min_samples_leaf
        #self.trees = ...

    def predict(self, X_test) -> np.ndarray:
        
        prediction_leaves=[]
        #weighted_predictions=[]
        #count_leaves=[]
        list_y_vals=[]
        final_list=[]

        for tree in self.trees:

            leaves=tree.predict(X_test)
            prediction_leaves.append(leaves)
            
        

        for i in range(self.n_estimators):

            list_y_vals.append([leaf.y_vals for leaf in prediction_leaves[i]])
            
        for j in range(len(X_test)):


            final_list.append(np.concatenate([list_y_vals[i][j] for i in range(self.n_estimators)]))
            
           
        return np.array([stats.mode(final_list[i])[0][0] for i in range(len(X_test))])
        
        
    def score(self, X_test, y_test) -> float:
        """
        Given a 2D nxp X_test array and 1D nx1 y_test array with one or more records,
        collect the predicted class for each record and then compute accuracy between
        that and y_test.
        """
        
        y_pred = self.predict(X_test)
        return accuracy_score(y_test, y_pred)
        

In [964]:
import numpy as np
from sklearn.datasets import \
    load_boston, load_iris, load_diabetes, load_wine, \
    load_breast_cancer, fetch_california_housing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
import inspect

#X, y = load_boston(return_X_y=True)

In [965]:
stats.mode([1,1,2,2,2])[0][0]

2

In [966]:

#X, y = load_wine(return_X_y=True)
X, y = load_boston(return_X_y=True)
#X, y = load_diabetes(return_X_y=True)


In [967]:
X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=0.20)

In [968]:
min_samples_leaf=3
max_features=0.3
oob=True
n_estimators=18


In [969]:
#rf = RandomForestClassifier621(n_estimators=n_estimators, min_samples_leaf=min_samples_leaf, max_features=max_features, oob_score=oob)
rf= RandomForestRegressor621(n_estimators=n_estimators, min_samples_leaf=min_samples_leaf, max_features=max_features, oob_score=oob)

In [970]:
rf.fit(X_train, y_train)

/Users/akshaypamnani/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/akshaypamnani/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/akshaypamnani/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/akshaypamnani/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/akshaypamnani/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/akshaypamnani/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_

In [971]:
len(X_train)

404

In [972]:
rf.oob_score_

0.8050437805213196

In [962]:
arr=np.array(test_lis)

In [788]:
arr[np.where(~np.isnan(test_lis))[0]]


array([13.31844778, 29.58809524, 34.93125   , 18.87104377, 22.88666667,
       16.97      , 15.75647118, 20.23875   , 22.60888889, 11.94300977,
       10.32252976, 21.55833333, 23.0475    , 31.53388889, 15.06111111,
       24.24220238, 15.35588235, 24.71866667, 46.75833333, 20.24266667,
       33.16857143, 22.521875  , 10.24708333, 17.77708333, 27.25277778,
       18.89194444, 25.99919643, 25.175     , 19.943     , 16.59307692,
       19.91547619, 17.421875  , 16.25066667, 14.93731041, 18.84444444,
       12.8127381 , 21.35708333, 20.85833333, 21.53222222, 23.32404762,
       22.60777778, 20.86283333, 20.87645833, 28.95944444, 20.1743075 ,
       36.52666667, 14.29068783, 26.84166667, 25.55431973, 23.47733333,
       35.84541667, 17.34495238, 21.82642857, 18.99916667, 18.46233333,
       20.96154321, 20.695     , 15.885     ,  8.205     , 37.71142857,
       16.19161111, 23.88541667, 14.65444919, 25.38772727, 18.89333333,
       31.31166667, 22.90805556, 20.4055102 , 25.46566667, 23.94

In [411]:
test_lis[0]

13.318447783447784

In [434]:
test_lis[lis]
        #y_filtered=y[np.where(~np.isnan(oob_predictions))]

TypeError: list indices must be integers or slices, not list

In [424]:
indices=np.where(~np.isnan(test_lis))[0]


In [365]:
rf.trees[0].oob_index

array([  0,   2,   4,   6,  12,  13,  15,  18,  28,  33,  35,  40,  41,
        42,  48,  50,  51,  52,  54,  59,  62,  63,  64,  65,  66,  69,
        70,  71,  74,  75,  76,  78,  79,  80,  81,  93,  94,  97,  98,
        99, 100, 103, 104, 105, 107, 109, 110, 112, 119, 122, 123, 125,
       130, 131, 137, 145, 146, 152, 154, 156, 158, 160, 161, 165, 166,
       171, 184, 185, 189, 193, 196, 199, 200, 203, 208, 209, 213, 215,
       220, 225, 231, 235, 237, 239, 241, 244, 251, 258, 261, 263, 273,
       274, 275, 279, 280, 281, 284, 285, 286, 287, 289, 292, 294, 298,
       299, 300, 303, 307, 310, 311, 313, 315, 316, 317, 319, 323, 327,
       330, 333, 334, 340, 341, 342, 343, 344, 346, 348, 349, 351, 353,
       357, 358, 359, 361, 368, 370, 374, 377, 379, 385, 387, 393, 398,
       401])

In [355]:
rf.compute_oob_score(X_train,y_train)

TypeError: 'int' object is not iterable

In [345]:
leaves_arr=rf.predict(X_test)






In [346]:
leaves_arr_2=rf.predict2(X_test)

(2, 89)


In [347]:
leaves_arr

array([138.1       ,  87.16438356,  99.48684211, 115.46153846,
       147.55555556,  92.89230769, 175.05263158, 116.07042254,
       100.05970149, 112.45945946, 102.08695652, 112.68055556,
       233.13333333,  82.51898734, 129.32467532, 186.97435897,
        85.13432836,  89.05714286, 133.64179104, 162.72972973,
       238.05555556, 241.63888889, 224.61643836, 124.59722222,
       187.90140845, 136.2987013 , 167.23287671,  98.84415584,
       112.16666667, 124.91044776, 207.66666667, 155.17647059,
       125.33333333,  73.44285714, 109.04      , 186.625     ,
       142.19117647, 160.04166667, 152.32432432, 213.01538462,
       111.62025316, 224.38157895,  95.05633803, 231.51470588,
       172.0375    , 151.57534247, 101.49333333, 186.98684211,
       232.0625    , 143.59036145, 101.79365079, 193.87837838,
       106.63013699,  94.35483871, 205.86956522, 208.14516129,
       145.87878788, 107.42465753, 173.27272727, 168.20895522,
       191.625     , 170.87671233,  91.36619718, 178.60

In [348]:
leaves_arr_2

array([28.4742268 , 20.86229508, 21.12011173, 24.87845304, 34.94736842,
       24.44534413, 35.95675676, 26.84364821, 25.4904943 , 23.77714286,
       24.89045936, 27.04333333, 53.8       , 17.86027397, 28.69740634,
       40.96629213, 21.85440613, 21.34931507, 32.7985348 , 37.86792453,
       56.38157895, 57.60927152, 52.38658147, 29.12662338, 45.22372881,
       30.42028986, 37.79566563, 20.5148248 , 26.56578947, 31.82129278,
       51.35842294, 38.51094891, 30.99641577, 17.72758621, 24.4119403 ,
       44.49337748, 35.81111111, 38.41      , 34.79012346, 56.05668016,
       24.29201102, 50.45266272, 22.57190635, 57.87867647, 36.99731183,
       35.57877814, 23.13677812, 42.0443787 , 62.93220339, 28.58033573,
       27.76190476, 44.834375  , 24.09907121, 26.59090909, 50.91397849,
       58.13063063, 38.512     , 25.54397394, 38.89795918, 42.85171103,
       46.61148649, 39.6       , 21.69565217, 42.41137124, 30.34219269,
       35.18518519, 23.52531646, 46.69387755, 44.54054054, 45.56

In [ ]:
lis

In [292]:
final_list=[]

for j in range(len(X_test)):

    final_list.append(np.concatenate([leaves_arr[i][j] for i in range(n_estimators)]))


IndexError: invalid index to scalar variable.

In [282]:
[stats.mode(final_list[i])[0][0] for i in range(len(X_test))]

[1,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 1,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 1]

In [256]:
len(lis1)

18

In [257]:
list_y_vals=[]

for i in range(n_estimators):

    list_y_vals.append([(leaf.y_vals) for leaf in leaves_arr[i]])

AttributeError: 'numpy.ndarray' object has no attribute 'y_vals'

In [258]:
test_arr=np.array(list_y_vals)

In [259]:
test_arr

array([], dtype=float64)

In [137]:
lis=[]

for i in range(len(X_test)):

    lis.append(np.concatenate(test_arr[:,i]))

In [140]:
final_arr=np.array(lis)

/var/folders/_g/93vdh72j2bg48859p9w3jd1w0000gn/T/ipykernel_71778/1629953685.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  final_arr=np.array(lis)


In [148]:
for i in 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0])

In [141]:
final_arr.shape

(36,)

In [142]:
final_arr

array([array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
              2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
              2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
              2, 2, 2, 2, 2, 2, 2, 2])                                         ,
       array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 0, 1, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2,
              1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 0,
              0, 0, 0, 0, 0])                                                  ,
       array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])     ,
       array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1,

In [89]:
leaves_ele_list

[array([2, 2, 2, 2, 2]),
 array([0, 0, 0]),
 array([1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1]),
 array([0, 0, 0]),
 array([2, 2, 2, 2]),
 array([1, 1, 1, 1]),
 array([0, 0, 0]),
 array([2, 2, 2, 2]),
 array([0, 0, 0]),
 array([1, 1, 1, 1, 1, 1]),
 array([0, 0, 0]),
 array([2, 2, 2, 2]),
 array([0, 0, 0]),
 array([0, 0, 0]),
 array([1, 1, 1, 1, 1]),
 array([0, 0, 0]),
 array([2, 2, 2, 2]),
 array([0, 0, 0]),
 array([0, 0, 0]),
 array([0, 0, 0]),
 array([1, 1, 1, 1, 1, 1]),
 array([1, 1, 1, 1, 1, 1]),
 array([0, 0, 0]),
 array([0, 0, 0]),
 array([1, 1, 1, 1, 1, 1]),
 array([2, 2, 2, 2, 2]),
 array([1, 1, 1, 1]),
 array([1, 2, 0, 1]),
 array([0, 0, 0]),
 array([0, 0, 0]),
 array([0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 1]),
 array([0, 0, 0]),
 array([0, 0, 0, 0, 0]),
 array([2, 1, 1, 2])]

In [92]:
np.shape(np.array(list_y_vals))

/var/folders/_g/93vdh72j2bg48859p9w3jd1w0000gn/T/ipykernel_71778/587811388.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.shape(np.array(list_y_vals))


(18, 36)

In [100]:
stats.mode(np.concatenate(list_y_vals[0]))[0][0]

0

In [103]:
stats.mode([1,1,2,2])[0][0]

1